# rGreat local vs greatpy

This notebook allows you to compare results between local rGreat and enrichment function from greatpy. 

r-base == 3.6.1

python version == 3.8 

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import rpy2
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects import r as r
pandas2ri.activate()
import pandas as pd

import os 
import matplotlib.pyplot as plt
import greatpy as great 
from scipy.stats import pearsonr
import re
import time 
import seaborn as sns 
import numpy as np

In [5]:
rpy2.__version__

'3.5.2'

In [6]:
importr('rGREAT')
ranges = importr('GenomicRanges')

In [7]:
def rgreat_online_vs_local_vs_greatpy(): 
    stat_df = {
        "name" : [],
        "pearson_binom" : [], 
        "pearson_hypergeom" : []
    }
    pp = {
        "name" : [],
        "before_pp_greatpy_size" : [], 
        "before_pp_local_size" : [],
        "final_size" : [],
        "%_of_diffrent_GO_term" : [],
    }
    # test_file = ["01_random.bed","06_height_snps_hg38.bed","07_height_snps_hg19.bed","10_MAX.bed"]
    test_file = ["06_height_snps_hg38.bed","10_MAX.bed"]

    # test_file = ["06_height_snps_hg38.bed"]

    for name in test_file : 
        # find the assembly 
        if re.match(".*hg19.*",name) != None : 
            assembly = "hg19"
        else : 
            assembly = "hg38"

        # online test 
        # res_online = rpy2.robjects.r['submitGreatJob'](f"../../data/tests/test_data/input/{name}",species=f"{assembly}",help=False)
        # res_online = rpy2.robjects.r['getEnrichmentTables'](res_online)

        # time.sleep(30)

        # local test
            # proprocessing : make a Grange frame 
        df = r["read.csv"](f"../../data/tests/test_data/input/{name}",sep='\t')
        seqname = rpy2.robjects.StrVector(["seqnames", "seqname","chromosome", "X.Chr","chr", "chromosome_name","seqid"])
        end = rpy2.robjects.StrVector(['end', 'stop'])
        df = ranges.makeGRangesFromDataFrame(df,seqnames_field=seqname)

            # great calculation
        local = rpy2.robjects.r['great'](df, "msigdb:C5", f"GREAT:{assembly}",verbose=False)
        local = rpy2.robjects.r['getEnrichmentTables'](local)

        time.sleep(30)

        # greatpy calculation
        # greatpy = great.tl.GREAT.enrichment(
        #     test_file=f"../../data/tests/test_data/input/{name}",
        #     regdom_file=f"../../data/human/{assembly}/regulatory_domain.bed",
        #     chr_size_file=f"../../data/human/{assembly}/chr_size.bed",
        #     annotation_file=f"../../data/human/ontologies.csv",
        #     binom=True,
        #     hypergeom=True,
        #     )
        # time.sleep(30)

        # create each dataframe
            # online
        # name = [cdc.lower() for cdc in list(res_online.rx2("GO Molecular Function").rx2("name"))+list(res_online.rx2("GO Biological Process").rx2("name"))+list(res_online.rx2("GO Cellular Component").rx2("name"))]
        # online = pd.DataFrame({
        #     "id" : list(res_online.rx2("GO Molecular Function").rx2("ID"))+list(res_online.rx2("GO Biological Process").rx2("ID"))+list(res_online.rx2("GO Cellular Component").rx2("ID")),
        #     "name" : name,
        #     "binom_p_val" : list(res_online.rx2("GO Molecular Function").rx2("Binom_Raw_PValue"))+list(res_online.rx2("GO Biological Process").rx2("Binom_Raw_PValue"))+list(res_online.rx2("GO Cellular Component").rx2("Binom_Raw_PValue")),
        #     "hyper_p_val" : list(res_online.rx2("GO Molecular Function").rx2("Hyper_Raw_PValue"))+list(res_online.rx2("GO Biological Process").rx2("Hyper_Raw_PValue"))+list(res_online.rx2("GO Cellular Component").rx2("Hyper_Raw_PValue"))
        #     })

            # local
        name_go = list(local.rx2("id")) 
        name_go = [" ".join(cdc.lower().split("_")[1:]) for cdc in list(local.rx2("id"))]
        local = pd.DataFrame({
            "name": name_go,
            "binom_p_val" : list(local.rx2("p_value")),
            "hyper_p_val" : list(local.rx2("p_value_hyper"))
            })

        ann = pd.read_csv("../../data/human/ontologies.csv",sep=';',comment='#',header=0,)
        ann["name"] = ann['name'].str.lower()
        go = []
        for i in range(local.shape[0]) : 
            curr = local.iloc[i]

            if ann.loc[ann["name"].isin([curr["name"]])].shape[0]>0 : 
                # print(ann.loc[ann["name"].isin([curr["name"]])].iloc[0]["id"])
                go.append(ann.loc[ann["name"].isin([curr["name"]])].iloc[0]["id"])
            else :
                go.append("")
        local["id"] = go 
        local = local.loc[local["id"]!=""]
        return local 
            # greatpy
        greatpy["go_term"] = greatpy["go_term"].str.lower()

        # correlation between online and greatpy 
        online = online[online["id"].isin(greatpy["id"])]
        greatpy = greatpy[greatpy["id"].isin(online["id"])]
        res_bin=[]
        res_hyp=[]
        cols_to_supr = []
        for i in range(len(online)) : 
            curr_o = online.iloc[i]
            if greatpy.loc[greatpy["id"] == curr_o["id"]].shape[0] > 0 : 
                res_bin.append( float(greatpy.loc[greatpy["id"] == curr_o["id"]]["binom_p_value"]) )
                res_hyp.append( float( greatpy.loc[greatpy["id"] == curr_o["id"]]["hypergeom_p_value"] ))
            else : 
                cols_to_supr.append(i)
        online["binom_p_value_greatpy"] = -np.log(res_bin)
        online["hypergeom_p_value_greatpy"] = -np.log(res_hyp)
        online = online.drop(cols_to_supr,axis=1)

        # correlation between local and online 
        online = online[online["id"].isin(local["id"])]
        local = local[local["id"].isin(online["id"])]
        res_bin = []
        res_hyp = []
        cols_to_supr = []
        for i in range(len(online)) : 
            curr_o = online.iloc[i]
            if local.loc[local["id"] == curr_o["id"]].shape[0] > 0 : 
                res_bin.append( float(local.loc[local["id"] == curr_o["id"]]["binom_p_value"]) )
                res_hyp.append( float( local.loc[local["id"] == curr_o["id"]]["hypergeom_p_value"] ))
            else : 
                cols_to_supr.append(i)
        online["binom_p_value_local"] = -np.log(res_bin)
        online["hypergeom_p_value_local"] = -np.log(res_hyp)
        online = online.drop(cols_to_supr,axis=1)

        res = {"name1":[],"name2":[],"value":[]}

        res["name1"].append("online")
        res["name2"].append("greatpy")
        res["value"].append(pearsonr(online["binom_p_value"],online["binom_p_value_greatpy"])[0])

        res["name1"].append("greatpy")
        res["name2"].append("online")
        res["value"].append(pearsonr(online["binom_p_value_greatpy"],online["binom_p_value"])[0])

        res["name1"].append("online")
        res["name2"].append("local")
        res["value"].append(pearsonr(online["binom_p_value"],online["binom_p_value_local"])[0])

        res["name1"].append("local")
        res["name2"].append("online")
        res["value"].append(pearsonr(online["binom_p_value"],online["binom_p_value_local"])[0])

        res["name1"].append("online")
        res["name2"].append("online")
        res["value"].append(pearsonr(online["binom_p_value"],online["binom_p_value"])[0])

        res["name1"].append("local")
        res["name2"].append("greatpy")
        res["value"].append(pearsonr(online["binom_p_value_greatpy"],online["binom_p_value_local"])[0])

        res["name1"].append("greatpy")
        res["name2"].append("local")
        res["value"].append(pearsonr(online["binom_p_value_local"],online["binom_p_value_local"])[0])

        res["name1"].append("local")
        res["name2"].append("local")
        res["value"].append(pearsonr(online["binom_p_value_local"],online["binom_p_value_local"])[0])


        test = pd.DataFrame(res)
        test = test.pivot("name1","name2","value")
        test.replace(np.nan,1,inplace=True)
        test

        sns.clustermap(test,figsize=(5,5))


        time.sleep(90) # just to have not a burning computer ^^ 
        
    

In [8]:
# local,online,greatpy = rgreat_online_vs_local_vs_greatpy()

R[write to console]: The default enrichment tables contain no associated genes for the input
regions.You can set `download_by = 'tsv'` to download the complete
table,but note only the top 500 regions can be retreived. See the
following link:

https://great-help.atlassian.net/wiki/spaces/GREAT/pages/655401/Export#Export-GlobalExport

R[write to console]: * use genome 'hg38'.

R[write to console]: * TSS extension mode is 'basalPlusExt'.

R[write to console]: * construct the basal domains by extending 5000bp to upstream and 1000bp to downsteram of TSS.

R[write to console]: * calculate distances to neighbour regions.

R[write to console]: * extend to both sides until reaching the neighbour genes or to the maximal extension (1000000bp).



KeyError: 'id'

: 

In [ ]:
rgreat_online_vs_local_vs_greatpy()